In [1]:
from sqlalchemy.orm import scoped_session, sessionmaker

# Create a session that flushes before each query but never commits.
# Also wrap it in a scoped_session to ensure we use different sessions for different threads.
DBSession = scoped_session(sessionmaker(
    autoflush=True, autocommit=False
))

In [2]:
from sqlalchemy.ext.declarative import declarative_base

# Create a declarative base class from which our models can inherit
# Also takes charge of creating a database metadata object for us.
DeclarativeBase = declarative_base()

In [3]:
from sqlalchemy import Column, Integer, DateTime, Unicode
from datetime import datetime

class Person(DeclarativeBase):
    __tablename__ = 'persons'
    
    uid = Column(Integer, primary_key=True)
    name = Column(Unicode(255), nullable=False)
    
    def __repr__(self):
        return '<Person: %s>' % self.name.encode('ascii', errors='ignore')

In [4]:
from sqlalchemy import ForeignKey
from sqlalchemy.orm import relationship

class Log(DeclarativeBase):
    __tablename__ = 'logs'

    uid = Column(Integer, primary_key=True)
    timestamp = Column(DateTime, nullable=False, default=datetime.utcnow)
    
    person_id = Column(Integer, ForeignKey(Person.uid))
    person = relationship(Person, backref="logs")
    
    def __repr__(self):
        return '<Log: %s, %s>' % (self.timestamp, self.person and self.person.name.encode('ascii', errors='ignore'))

In [5]:
from sqlalchemy import create_engine

engine = create_engine('sqlite:///:memory:')
DBSession.configure(bind=engine)

In [6]:
DeclarativeBase.metadata.create_all(engine)

In [7]:
p = Person(name=u'user')
DBSession.add(p)
DBSession.commit()

In [8]:
user = DBSession.query(Person).first()
print user

<Person: user>


In [9]:
l = Log(person=user)
DBSession.add(l)
DBSession.commit()

In [10]:
log = DBSession.query(Log).first()
print log

<Log: 2015-10-16 10:22:21.191837, user>


In [11]:
DBSession.delete(user)
DBSession.commit()

In [12]:
print log

<Log: 2015-10-16 10:22:21.191837, None>


In [13]:
class OtherLog(DeclarativeBase):
    __tablename__ = 'otherlogs'

    uid = Column(Integer, primary_key=True)
    timestamp = Column(DateTime, nullable=False, default=datetime.utcnow)
    
    person_id = Column(Integer, ForeignKey(Person.uid), nullable=False)
    person = relationship(Person, backref="otherlogs")
    
    def __repr__(self):
        return '<Log: %s, %s>' % (self.timestamp, self.person.name.encode('ascii', errors='ignore'))

DeclarativeBase.metadata.create_all(engine)

In [14]:
p = Person(name=u'user')
DBSession.add(p)

olog = OtherLog(person=p)
DBSession.add(olog)

DBSession.commit()

In [15]:
print olog

<Log: 2015-10-16 10:22:22.501156, user>


In [16]:
DBSession.delete(p)

try:
    DBSession.commit()
except Exception as e:
    DBSession.rollback()
    print e

(sqlite3.IntegrityError) NOT NULL constraint failed: otherlogs.person_id [SQL: u'UPDATE otherlogs SET person_id=? WHERE otherlogs.uid = ?'] [parameters: (None, 1)]


In [17]:
DBSession.query(OtherLog).delete()

1

In [18]:
from sqlalchemy.orm import backref


class MoreLog(DeclarativeBase):
    __tablename__ = 'morelogs'

    uid = Column(Integer, primary_key=True)
    timestamp = Column(DateTime, nullable=False, default=datetime.utcnow)
    
    person_id = Column(Integer, ForeignKey(Person.uid), nullable=False)
    person = relationship(Person, backref=backref("morelogs", cascade='all, delete-orphan'))
    
    def __repr__(self):
        return '<Log: %s, %s>' % (self.timestamp, self.person.name.encode('ascii', errors='ignore'))

DeclarativeBase.metadata.create_all(engine)

In [19]:
p = Person(name=u'user')
DBSession.add(p)

mlog = MoreLog(person=p)
DBSession.add(mlog)

DBSession.commit()

In [20]:
print DBSession.query(MoreLog).all()

[<Log: 2015-10-16 10:22:24.821074, user>]


In [21]:
DBSession.delete(p)
DBSession.commit()

In [22]:
print DBSession.query(MoreLog).all()

[]


In [23]:
p = Person(name=u'user')
DBSession.add(p)

mlog = MoreLog(person=p)
DBSession.add(mlog)

DBSession.commit()

In [24]:
mlog = DBSession.query(MoreLog).first()

DBSession.delete(mlog)
DBSession.commit()

In [25]:
print DBSession.query(Person).all()

[<Person: user>, <Person: user>]


In [26]:
from sqlalchemy.orm import backref


class OneLog(DeclarativeBase):
    __tablename__ = 'onelogs'

    uid = Column(Integer, primary_key=True)
    timestamp = Column(DateTime, nullable=False, default=datetime.utcnow)
    
    person_id = Column(Integer, ForeignKey(Person.uid), nullable=False)
    person = relationship(Person, backref=backref("log", uselist=False, cascade='all, delete-orphan'))
    
    def __repr__(self):
        return '<Log: %s, %s>' % (self.timestamp, self.person.name.encode('ascii', errors='ignore'))

DeclarativeBase.metadata.create_all(engine)

In [27]:
p = Person(name=u'user')
DBSession.add(p)

olog = OneLog(person=p)
DBSession.add(olog)

DBSession.commit()

In [28]:
print DBSession.query(OneLog).all()

[<Log: 2015-10-16 10:22:35.636799, user>]


In [29]:
olog.person

<Person: user>

In [30]:
DBSession.delete(p)
DBSession.commit()

In [31]:
print DBSession.query(OneLog).all()

[]


In [32]:
p = Person(name=u'user')
DBSession.add(p)

olog = OneLog(person=p)
DBSession.add(olog)

DBSession.commit()

In [33]:
DBSession.query(Person).filter_by(name=u'user').delete()

3

In [34]:
l = DBSession.query(OneLog).first()
print l.person

None


In [35]:
from sqlalchemy import Table
from sqlalchemy.orm import backref


manylog_person_table = Table('manylog_person', DeclarativeBase.metadata,
   Column('person_id', Integer,
          ForeignKey('persons.uid', onupdate="CASCADE", ondelete="CASCADE"),
          primary_key=True),
   Column('manylog_id', Integer,
          ForeignKey('manylogs.uid', onupdate="CASCADE", ondelete="CASCADE"),
          primary_key=True)
)



class ManyLog(DeclarativeBase):
    __tablename__ = 'manylogs'

    uid = Column(Integer, primary_key=True)
    timestamp = Column(DateTime, nullable=False, default=datetime.utcnow)
    
    persons = relationship(Person, secondary=manylog_person_table, backref=backref("manylogs"))
    
    def __repr__(self):
        return '<Log: %s, %s>' % (self.timestamp, ','.join(p.name.encode('ascii', errors='ignore') for p in self.persons))

    
DeclarativeBase.metadata.create_all(engine)

In [36]:
person1 = Person(name=u'person1')
person2 = Person(name=u'person2')
DBSession.add(person1)
DBSession.add(person2)

mlog = ManyLog(persons=[person1, person2])
DBSession.add(mlog)

DBSession.commit()

In [37]:
print DBSession.query(ManyLog).all()

[<Log: 2015-10-16 10:22:48.469913, person1,person2>]


In [38]:
p = DBSession.query(Person).filter_by(name='person1').first()
DBSession.delete(p)
DBSession.commit()

/Users/amol/venv/notebook/lib/python2.7/site-packages/sqlalchemy/sql/sqltypes.py:185: SAWarning: Unicode type received non-unicode bind param value 'person1'. (this warning may be suppressed after 10 occurrences)
  (util.ellipses_string(value),))


In [39]:
print DBSession.query(ManyLog).all()

[<Log: 2015-10-16 10:22:48.469913, person2>]


In [40]:
p = DBSession.query(Person).filter_by(name='person2').first()
DBSession.delete(p)
DBSession.commit()

/Users/amol/venv/notebook/lib/python2.7/site-packages/sqlalchemy/sql/sqltypes.py:185: SAWarning: Unicode type received non-unicode bind param value 'person2'. (this warning may be suppressed after 10 occurrences)
  (util.ellipses_string(value),))


In [41]:
print DBSession.query(ManyLog).all()

[<Log: 2015-10-16 10:22:48.469913, >]


In [42]:
DBSession.query(manylog_person_table).all()

[]

In [43]:
from sqlalchemy import Table
from sqlalchemy.orm import backref


manylogdel_person_table = Table('manylogdel_person', DeclarativeBase.metadata,
   Column('person_id', Integer,
          ForeignKey('persons.uid', onupdate="CASCADE", ondelete="CASCADE"),
          primary_key=True),
   Column('manylogdel_id', Integer,
          ForeignKey('manylogsdel.uid', onupdate="CASCADE", ondelete="CASCADE"),
          primary_key=True)
)



class ManyLogDeletable(DeclarativeBase):
    __tablename__ = 'manylogsdel'

    uid = Column(Integer, primary_key=True)
    timestamp = Column(DateTime, nullable=False, default=datetime.utcnow)
    
    persons = relationship(Person, secondary=manylogdel_person_table, backref=backref("manylogsdel", cascade='all, delete-orphan'))
    
    def __repr__(self):
        return '<Log: %s, %s>' % (self.timestamp, ','.join(p.name.encode('ascii', errors='ignore') for p in self.persons))

    
DeclarativeBase.metadata.create_all(engine)

In [44]:
try:
    person1 = Person(name='person1')
except Exception as e:
    print e

On Person.manylogsdel, delete-orphan cascade is not supported on a many-to-many or many-to-one relationship when single_parent is not set.   Set single_parent=True on the relationship().
